In [0]:
#imports 

from datetime import datetime
from pyspark.sql.functions import current_timestamp, col, expr


In [0]:

# Informações Unity Catalog

catalogo =  "fiscaliza_legislativo"
schemaDestino = "silver"
schemaOrigem = "bronze"
tabela = "votacoes_proposicoes"

TABELA_SILVER = f"{catalogo}.{schemaDestino}.{tabela}"


# Inicia execução

In [0]:
TInicio = datetime.now()
print(TInicio)

In [0]:
#Cria Dataframe da tabela Bronze

df = spark.sql(f"SELECT * FROM {catalogo}.{schemaOrigem}.{tabela}")


In [0]:
# Verifica % de nulos de Nulos

#df.toPandas().isnull().sum() / df.count()

In [0]:
#Seleciona Colunas da Tabela
df = df.select("idVotacao", "proposicao_id", "data", "descricao")

In [0]:
# Transforma coluna data em tipo Date

df = df.withColumn('data', expr("try_to_date(data, 'yyyy-MM-dd')"))

In [0]:
#Transforma os tipos de colunas 

df = df.withColumn('proposicao_id', expr("try_cast(proposicao_id AS INT)"))

In [0]:
#Remove Votações com proposicoes nulas

df = df.where(df.proposicao_id.isNotNull())

In [0]:
#Inclui data de ingestão na camada silver
df = df.withColumn('_data_ingestao', current_timestamp())
     

In [0]:
#salva Tabela no catalogo

df.write.format("delta") \
              .mode('overwrite') \
              .option("overwriteSchema", "true") \
              .saveAsTable(TABELA_SILVER)

In [0]:
TFim = datetime.now()
print(TFim)
print("Duração:", TFim - TInicio)